### Creating a logistic regression to classify Lyon vs. Monaco

- 6th places vs 7th


In [58]:
import numpy as np
import pandas as pd
from statsbombpy import sb
pd.set_option('display.max_columns', None)
from mplsoccer import Pitch, Sbopen
from mplsoccer import VerticalPitch,Pitch
import networkx as nx
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import networkx as nx
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import math 


In [59]:
def digraph(data):
        G = nx.DiGraph()
        pass_counts = {}
        players = data["player_name"].unique()
        G.add_nodes_from(players)
        for _, row in data.iterrows():
                key = (row["player_name"], row["pass_recipient_name"])
                if key in pass_counts:
                        pass_counts[key] += 1
                else:
                        pass_counts[key] = 1

        for key, weight in pass_counts.items():
                G.add_edge(key[0], key[1], weight=weight)
        
        return G

In [60]:
parser = Sbopen()
df_competitions = parser.competition()
df_competitions

,competition_id,season_id,country_name,competition_name,competition_gender,competition_youth,competition_international,season_name,match_updated,match_updated_360,match_available_360,match_available
0,9,27,Germany,1. Bundesliga,male,False,False,2015/2016,2023-08-17T23:51:11.837478,None,None,2023-08-17T23:51:11.837478
1,16,4,Europe,Champions League,male,False,False,2018/2019,2023-03-07T12:20:48.118250,2021-06-13T16:17:31.694,None,2023-03-07T12:20:48.118250
2,16,1,Europe,Champions League,male,False,False,2017/2018,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2021-01-23T21:55:30.425330
3,16,2,Europe,Champions League,male,False,False,2016/2017,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
4,16,27,Europe,Champions League,male,False,False,2015/2016,2021-08-27T11:26:39.802832,2021-06-13T16:17:31.694,None,2020-07-29T05:00
...,...,...,...,...,...,...,...,...,...,...,...,...
62,55,43,Europe,UEFA Euro,male,False,True,2020,2023-02-24T21:26:47.128979,2023-04-27T22:38:34.970148,2023-04-27T22:38:34.970148,2023-02-24T21:26:47.128979
63,35,75,Europe,UEFA Europa League,male,False,False,1988/1989,2023-06-18T19:28:39.443883,2021-06-13T16:17:31.694,None,2023-06-18T19:28:39.443883
64,53,106,Europe,UEFA Women's Euro,female,False,True,2022,2023-07-17T21:19:03.032991,2023-07-17T21:21:56.497106,2023-07-17T21:21:56.497106,2023-07-17T21:19:03.032991
65,72,107,International,Women's World Cup,female,False,True,2023,2023-09-01T12:34:19.705316,2023-09-01T12:35:45.762196,2023-09-01T12:35:45.762196,2023-09-01T12:34:19.705316


In [66]:
teams = parser.match(competition_id=7, season_id=27)
teams

,match_id,match_date,kick_off,home_score,away_score,match_status,match_status_360,last_updated,last_updated_360,match_week,competition_id,country_name,competition_name,season_id,season_name,home_team_id,home_team_name,home_team_gender,home_team_group,home_team_country_id,home_team_country_name,home_team_managers_id,home_team_managers_name,home_team_managers_nickname,home_team_managers_dob,home_team_managers_country_id,home_team_managers_country_name,away_team_id,away_team_name,away_team_gender,away_team_group,away_team_country_id,away_team_country_name,away_team_managers_id,away_team_managers_name,away_team_managers_nickname,away_team_managers_dob,away_team_managers_country_id,away_team_managers_country_name,metadata_data_version,metadata_shot_fidelity_version,metadata_xy_fidelity_version,competition_stage_id,competition_stage_name,stadium_id,stadium_name,stadium_country_id,stadium_country_name,referee_id,referee_name,referee_country_id,referee_country_name
0,3901178,2016-03-12,2016-03-12 18:00:00,1,1,available,unscheduled,2023-08-05 14:29:35.200486,NaT,30,7,France,Ligue 1,27,2015/2016,165,Lorient,male,None,78,France,2983.0,Sylvain Ripoll,Sylvain Ripoll,1971-08-15,78.0,France,147,Marseille,male,None,78,France,505.0,José Miguel González Martín del Campo,Míchel,1963-03-23,214.0,Spain,1.1.0,2,2,1,Regular Season,204.0,Stade Yves Allainmat - Le Moustoir,78.0,France,78.0,Ruddy Buquet,78.0,France
1,3901260,2016-05-14,2016-05-14 21:00:00,1,0,available,unscheduled,2023-08-03 18:34:31.678988,NaT,38,7,France,Ligue 1,27,2015/2016,165,Lorient,male,None,78,France,2983.0,Sylvain Ripoll,Sylvain Ripoll,1971-08-15,78.0,France,160,Gazélec Ajaccio,male,None,78,France,121.0,Thierry Laurey,Thierry Laurey,1964-02-17,78.0,France,1.1.0,2,2,1,Regular Season,204.0,Stade Yves Allainmat - Le Moustoir,78.0,France,92.0,Benoît Millot,78.0,France
2,3829470,2015-09-20,2015-09-20 17:00:00,2,0,available,unscheduled,2023-08-03 12:36:31.634292,NaT,6,7,France,Ligue 1,27,2015/2016,135,Saint-Étienne,male,None,78,France,281.0,Christophe Galtier,Christophe Galtier,1966-08-23,78.0,France,144,Nantes,male,None,78,France,438.0,Michel Der Zakarian,Michel Der Zakarian,1963-02-18,12.0,Armenia,1.1.0,2,2,1,Regular Season,4810.0,Stade Geoffroy-Guichard,78.0,France,92.0,Benoît Millot,78.0,France
3,3901259,2016-05-14,2016-05-14 21:00:00,1,1,available,unscheduled,2023-08-06 10:27:05.199641,NaT,38,7,France,Ligue 1,27,2015/2016,133,Troyes,male,None,78,France,1003442.0,Mohamed Bradja,Mohamed Bradja,1969-11-16,4.0,Algeria,147,Marseille,male,None,78,France,3004.0,Franck Passi,Franck Passi,1966-03-28,78.0,France,1.1.0,2,2,1,Regular Season,176.0,Stade de l''Aube,78.0,France,79.0,François Letexier,78.0,France
4,3901263,2016-05-14,2016-05-14 21:00:00,1,2,available,unscheduled,2023-08-06 22:42:28.958420,NaT,38,7,France,Ligue 1,27,2015/2016,134,Rennes,male,None,78,France,1001260.0,Rolland Courbis,Rolland Courbis,1953-08-12,78.0,France,2736,Bastia,male,None,78,France,3632.0,François Ciccolini,François Ciccolini,1962-06-03,78.0,France,1.1.0,2,2,1,Regular Season,NaN,NaN,NaN,NaN,83.0,Mikael Lesage,78.0,France
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
372,3829419,2015-08-08,2015-08-08 21:00:00,0,2,available,unscheduled,2023-08-01 22:11:16.865156,NaT,1,7,France,Ligue 1,27,2015/2016,139,Montpellier,male,None,78,France,1001260.0,Rolland Courbis,Rolland Courbis,1953-08-12,78.0,France,145,Angers,male,None,78,France,301.0,Stéphane Moulin,Stéphane Moulin,1967-08-04,78.0,France,1.1.0,2,2,1,Regular Season,165.0,Stade de la Mosson,78.0,France,87.0,Hakim Ben El Hadj Salem,78.0,France
373,3829418,2015-08-08,2015-08-08 21:00:00,1,0,available,unscheduled,2023-08-03 11:52:42.299524,NaT,1,7,France,Ligue 1,27,2015/2016,144,Nantes,male,None,78,France,438.0,Michel Der Zakarian,Michel Der Zakarian,1963-02-18,12.0,Armenia,142,Guingamp,male,None,78,Fra

In [67]:
df, related, freeze, tactics = parser.event(3901178)
events = sb.events(match_id=3901178)
events[events['shot_outcome'] == 'Goal']







,50_50,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,clearance_other,clearance_right_foot,counterpress,dribble_outcome,dribble_overrun,duel_outcome,duel_type,duration,foul_committed_card,foul_committed_offensive,foul_committed_type,foul_won_defensive,goalkeeper_body_part,goalkeeper_end_location,goalkeeper_outcome,goalkeeper_position,goalkeeper_punched_out,goalkeeper_technique,goalkeeper_type,id,index,interception_outcome,location,match_id,minute,miscontrol_aerial_won,off_camera,out,pass_aerial_won,pass_angle,pass_assisted_shot_id,pass_body_part,pass_cross,pass_cut_back,pass_deflected,pass_end_location,pass_goal_assist,pass_height,pass_inswinging,pass_length,pass_no_touch,pass_outcome,pass_outswinging,pass_recipient,pass_shot_assist,pass_straight,pass_switch,pass_technique,pass_through_ball,pass_type,period,play_pattern,player,player_id,position,possession,possession_team,possession_team_id,related_events,second,shot_aerial_won,shot_body_part,shot_deflected,shot_end_location,shot_first_time,shot_freeze_frame,shot_key_pass_id,shot_outcome,shot_statsbomb_xg,shot_technique,shot_type,substitution_outcome,substitution_replacement,tactics,team,team_id,timestamp,type,under_pressure
3640,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.448296,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,01748feb-b160-4f9b-b828-2c25f1ef25c4,1451,NaN,"[107.8, 35.4]",3901178,33,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,Regular Play,Abdul Majeed Waris,7615.0,Right Center Forward,81,Lorient,165,[9ede29bb-c351-4fe0-930e-6f6b0967bfc3],54,NaN,Left Foot,NaN,"[120.0, 37.0, 1.1]",NaN,"[{'location': [109.7, 40.0], 'player': {'id': ...",2a0947be-bf97-423b-8dd3-eb3b9bbe2d9a,Goal,0.400913,Normal,Open Play,NaN,NaN,NaN,Lorient,165,00:33:54.592,Shot,NaN
3645,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.645702,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,fe0a509f-578e-49f9-9db8-32245bd46542,1868,NaN,"[115.2, 52.4]",3901178,45,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,From Kick Off,Mauricio Aníbal Isla Isla,18874.0,Right Back,109,Marseille,147,"[283de354-c5f3-4da9-9bff-15ae8556d96c, 9da617c...",34,NaN,Right Foot,NaN,"[120.0, 40.7, 0.1]",True,"[{'location': [110.3, 31.9], 'player': {'id': ...",NaN,Goal,0.065786,Normal,Open Play,NaN,NaN,NaN,Marseille,147,00:00:34.425,Shot,True


In [57]:
df['match_outcome'] = 'Tie'  # Default to Te

df.loc[df['shot_outcome'] == 'Goal', 'match_outcome'] = 'Win'  # If there's a goal
df.loc[(df['possession_team'] != df['team']) & (df['shot_outcome'] == 'Goal'), 'match_outcome'] 

KeyError: 'shot_outcome'

##  Pseudo code
make data frame
Y- team with X1 = between centrality ...
- loop through df_match for each match
- if statement for same game just remove
    
- if away_team_name == Bayern Munich or Borussia Dortmund
    append team name
- make digraph

- compute centrality measures


In [65]:
def pass_data_monaco(match_id):
    df, related, freeze, tactics = parser.event(match_id)
    sub = df.loc[df["type_name"] == "Substitution"].loc[df["team_name"] == "AS Monaco"].iloc[0]["index"]

    mask_monaco = (df.type_name == 'Pass') & (df.team_name == "AS Monaco") & (df.index < sub) & (df.outcome_name.isnull()) & (df.sub_type_name != "Throw-in")
    df_pass = df.loc[mask_monaco, ['x', 'y', 'end_x', 'end_y', "player_name", "pass_recipient_name"]]
    df_pass["player_name"] = df_pass["player_name"].apply(lambda x: str(x).split()[-1])
    df_pass["pass_recipient_name"] = df_pass["pass_recipient_name"].apply(lambda x: str(x).split()[-1])
    return df_pass  

a = pass_data_monaco(3901178)
a

IndexError: single positional indexer is out-of-bounds

In [ ]:
def pass_data_lyon(match_id):
    df, related, freeze, tactics = parser.event(match_id)

    sub = df.loc[df["type_name"] == "Substitution"].loc[df["team_name"] == "Lyon"].iloc[0]["index"]
    mask_lyon = (df.type_name == 'Pass') & (df.team_name == "Lyon") & (df.index < sub) & (df.outcome_name.isnull()) & (df.sub_type_name != "Throw-in")
    df_pass = df.loc[mask_lyon, ['x', 'y', 'end_x', 'end_y', "player_name", "pass_recipient_name"]]
    df_pass["player_name"] = df_pass["player_name"].apply(lambda x: str(x).split()[-1])
    df_pass["pass_recipient_name"] = df_pass["pass_recipient_name"].apply(lambda x: str(x).split()[-1])
    return df_pass

In [ ]:
def avg(dictionary):
    total = sum(dictionary.values())

    average = total / len(dictionary)
    
    return average


NameError: name 'df_pass' is not defined

In [ ]:


def get_data(dat):
    data = pd.DataFrame()
    for index, row in dat.iterrows():
        if (row['home_team_name'] == 'AS Monaco'):
            match_id = row['match_id']
            passing_data_monaco = pass_data_monaco(match_id)
            
            G = digraph(passing_data_monaco)
            betweenness_centrality = avg(nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).values())
            katz_centrality = max(nx.katz_centrality(G).values())
            page_rank = max(nx.pagerank(G).values())
            
            match_data = {
                'Match ID': match_id,
                'Team': 'AS Monaco',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)
        if (row['away_team_name'] == 'AS Monaco'):
            match_id = row['match_id']
            passing_data_monaco = pass_data_monaco(match_id)
            G = digraph(passing_data_monaco)
            betweenness_centrality = avg(nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).values())
            katz_centrality = max(nx.katz_centrality(G).values())
            page_rank = max(nx.pagerank(G).values())
        
            match_data = {
                'Match ID': match_id,
                'Team': 'AS Monaco',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)

        if (row['home_team_name'] == 'Lyon'):
            match_id = row['match_id']
            passing_data_lyon = pass_data_lyon(match_id=match_id)
            G = digraph(passing_data_lyon)
            betweenness_centrality = avg(nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).values())
            katz_centrality = max(nx.katz_centrality(G).values())
            page_rank = max(nx.pagerank(G).values())
            
            
            match_data = {
                'Match ID': match_id,
                'Team': 'Lyon',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)
        if (row['away_team_name'] == 'Lyon'):
            match_id = row['match_id']
            passing_data_lyon = pass_data_lyon(match_id=match_id)
            G = digraph(passing_data_lyon)
            betweenness_centrality = avg(nx.betweenness_centrality(G))
            eigenvector_centrality = max(nx.eigenvector_centrality(G).values())
            katz_centrality = max(nx.katz_centrality(G).values())
            page_rank = max(nx.pagerank(G).values())
            
            match_data = {
                'Match ID': match_id,
                'Team': 'Lyon',
                'Betweenness Centrality': betweenness_centrality,
                'Eigenvector Centrality': eigenvector_centrality,
                'Katz Centrality': katz_centrality,
                'Page Rank': page_rank,
                'Average Clustering': (nx.average_clustering(G))
            }
            data = data.append(match_data, ignore_index=True)

    return data

d = get_data(dat=teams)
d


NameError: name 'teams' is not defined

In [ ]:
team_counts = d['Team'].value_counts()

print("Number of Lyon teams:", team_counts.get("Lyon", 0))
print("Number of AS Monaco teams:", team_counts.get("AS Monaco", 0))

Number of Lyon teams: 38
Number of AS Monaco teams: 38


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

In [ ]:
X = d.drop(['Team','Match ID'], axis=1)
y = d['Team']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=33)
model = LogisticRegression()
model.fit(X_train, y_train)


LogisticRegression()

In [ ]:
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
confusion = confusion_matrix(y_test, y_pred)
report = classification_report(y_test, y_pred)

In [ ]:
print(f"Accuracy: {accuracy * 100:.2f}%")
print("Confusion Matrix:\n", confusion)
print("Classification Report:\n", report)

Accuracy: 42.11%
Confusion Matrix:
 [[ 1 10]
 [ 1  7]]
Classification Report:
               precision    recall  f1-score   support

   AS Monaco       0.50      0.09      0.15        11
        Lyon       0.41      0.88      0.56         8

    accuracy                           0.42        19
   macro avg       0.46      0.48      0.36        19
weighted avg       0.46      0.42      0.32        19



In [ ]:
base_classifier = DecisionTreeClassifier(max_depth=6)  # You can adjust max_depth as needed

# Create the AdaBoost model
model = AdaBoostClassifier(base_classifier, n_estimators=20, random_state=33)

# Fit the model to the training data
model.fit(X_train, y_train)

# Make predictions on the test data
y_pred = model.predict(X_test)

# Calculate accuracyb
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

Accuracy: 0.5789473684210527


### can we predict whether a team is winning tied or losing based off the passing networks

1. create dataset
- create passing networks for english premier league
- create a loop with initial state == tied
- df_pass

In [ ]:
df[df['pass_goal_assist'] == True]

,id,index,period,timestamp,minute,second,possession,duration,match_id,type_id,type_name,possession_team_id,possession_team_name,play_pattern_id,play_pattern_name,team_id,team_name,tactics_formation,player_id,player_name,position_id,position_name,pass_recipient_id,pass_recipient_name,pass_length,pass_angle,pass_height_id,pass_height_name,end_x,end_y,body_part_id,body_part_name,sub_type_id,sub_type_name,x,y,under_pressure,outcome_id,outcome_name,out,counterpress,pass_cross,pass_switch,technique_id,technique_name,ball_recovery_recovery_failure,off_camera,block_deflection,pass_assisted_shot_id,pass_shot_assist,shot_statsbomb_xg,end_z,shot_key_pass_id,shot_first_time,goalkeeper_position_id,goalkeeper_position_name,aerial_won,substitution_replacement_id,substitution_replacement_name,foul_won_defensive,foul_committed_offensive,pass_goal_assist,pass_deflected,shot_deflected,foul_committed_card_id,foul_committed_card_name,dribble_overrun,pass_no_touch,pass_cut_back
1447,2a0947be-bf97-423b-8dd3-eb3b9bbe2d9a,1448,1,00:33:52.227000,33,52,81,0.597568,3901178,30,Pass,165,Lorient,1,Regular Play,165,Lorient,NaN,7766.0,Walid Mesloub,11.0,Left Defensive Midfield,7615.0,Abdul Majeed Waris,33.866207,0.722719,1.0,Ground Pass,102.6,38.9,40.0,Right Foot,NaN,NaN,77.2,16.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,108.0,Through Ball,NaN,NaN,NaN,01748feb-b160-4f9b-b828-2c25f1ef25c4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN
